In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from glob import glob
import os
import numpy as np
from classifier  import ConvNet, predict
from data import NIRCamDataset

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

dataset_path = '/data/scratch/sarperyurtseven/dataset/NIRCAM/1386/injections2/fc10_coron_injections_train_resized_unnormalized'
dataset = glob(os.path.join(dataset_path, '*.npy'))

nircam = NIRCamDataset(dataset, device)
nircam_loader = DataLoader(nircam, batch_size=512, shuffle=True)

model = ConvNet()
model.load_state_dict(torch.load('./models/model_20.pth'))
model.to(device)
model.eval()

ConvNet(
  (conv1): Conv2d(2, 32, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (conv2): Conv2d(32, 32, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (conv3): Conv2d(32, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (conv4): Conv2d(64, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (conv5): Conv2d(64, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
  (fc): Linear(in_features=1843200, out_features=2, bias=True)
  (activation): LeakyReLU(negative_slope=0.01)
  (softmax): Softmax(dim=1)
  (BN1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (BN2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout2d(p=0.4, inplace=False)
)

In [3]:
test = next(iter(nircam_loader))
results = predict(model, test[0])

In [2]:
x = torch.Tensor([[-0.5], [0.5], [0.2], [-0.3]])

In [9]:
t = torch.where(x < 0.5, x, 1)
t = torch.where(t > -0.5, t, -1)
t = torch.where((-0.5 < t) == (t < 0.5), 0, t)

t

tensor([[-1.],
        [ 1.],
        [ 0.],
        [ 0.]])

In [24]:
res = torch.where(x < 0, -1, 1)

In [26]:
test_class = torch.Tensor([[-1], [-1], [1], [-1]])

In [29]:
(test_class == res).sum().item()

3

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from torchvision import utils

def visTensor(kernels): 
    kernels = kernels[:,:1:,:]
    print(kernels.size())

    # normalize to (0,1) range so that matplotlib
    # can plot them
    kernels = kernels - kernels.min()
    kernels = kernels / kernels.max()
    filter_img = utils.make_grid(kernels, nrow = 8)
    # change ordering since matplotlib requires images to 
    # be (H, W, C)
    plt.imshow(filter_img.permute(1, 2, 0).cpu().numpy())

    # You can directly save the image as well using
    return kernels

In [3]:
test = next(iter(nircam_loader))

test[0].size()

torch.Size([512, 2, 120, 120])

In [5]:
np.min(test[0].cpu().numpy())

-1.0

In [ ]:
filter = model.conv1.weight.data.clone()
kernels = visTensor(filter)

plt.axis('off')
plt.ioff()
plt.show()

In [ ]:
filter[:,:1:,:].shape

In [9]:
dataset_path = '/data/scratch/sarperyurtseven/dataset/NIRCAM/1386/injections2/fc10_coron_injections_train_resized_unnormalized'
dataset = glob(os.path.join(dataset_path, '*.npy'))

dest_copy_path = '/data/scratch/bariskurtkaya/dataset/NIRCAM/1386/injections_24102023'
dataset[0]

'/data/scratch/sarperyurtseven/dataset/NIRCAM/1386/injections2/fc10_coron_injections_train_resized_unnormalized/jw01386-a3001_t004_nircam_f300m-maskrnd-sub320a335r-psf354-aug-rot2-flip0-vshift2-hshift0-vshiftp8-hshiftp10.npy'

In [14]:
import random
import shutil

random.shuffle(dataset)

In [15]:
train = dataset[:int(len(dataset)*0.8)]
test = dataset[int(len(dataset)*0.8):]

In [16]:
for file in train:
    shutil.copy(file, os.path.join(dest_copy_path, 'train'))

for file in test:
    shutil.copy(file, os.path.join(dest_copy_path, 'test'))